In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('data/webmd.csv')
df2 = pd.read_csv('data/thespruce.csv')
df3 = pd.read_csv('data/amazon.csv')
df = pd.concat([df1, df2, df3])
df = df.reset_index(drop=True)
columns_to_drop = ["KD", "CPC", "Traffic", "Current position", "Current URL inside", "Updated"]
df = df.drop(columns_to_drop, axis=1)
df = df.rename(columns={'Current URL': 'URL'})

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Keyword        90000 non-null  object
 1   SERP features  89313 non-null  object
 2   Volume         90000 non-null  int64 
 3   URL            90000 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.7+ MB


In [4]:
# remove rows  where kw contains non-ascii
df = df[df['Keyword'].apply(lambda x: all(ord(c) < 128 for c in x))]

In [5]:
kw_block_list = ["penis", "vagina", "sex", "slut", "dick", "fuck", "milf", "cum ", "cunt", "rape", "porn", "boner", "cock", "whore", "bitch", "futa", "wet dreams", "cowgirl position",  "reverse cowgirl", "girl squirt", "girls squirt", "squirting-orgasm", " squirt pee", "girl's body", "dildo", "butt-plug", "good head", "smells bad down there", "xxx", "aws.amazon", "amazon ", "am,azon","xxx", "wormwood", "fetish",  "vibrator", "sandalias de mujer"]

In [6]:
# remove blocklist kw
def remove_kw_blocklist(df, kw_blocklist):
    kw_pattern = '|'.join(kw_block_list)
    kw_mask = df['Keyword'].str.contains(kw_pattern, case=False)
    df = df[~kw_mask]
    return(df)

In [7]:
df = remove_kw_blocklist(df, kw_block_list)

In [8]:
url_block_list = ["aws.amazon", "www.amazon.com/kindle-dbs", "squirting-orgasm"]

In [9]:
# remove blocklist urls

# ****** DO THIS AFTER I HAVE TOP 20 URL LIST
def remove_url_blocklist(df, url_blocklist):
    url_pattern = '|'.join(url_block_list)
    url_mask = df['URL'].str.contains(url_pattern, case=False)
    df = df[~url_mask]
    return(df)

In [10]:
len(df)

88837

In [11]:
df.to_csv('data/combined.csv', index=False)

In [12]:
df.isna().sum()

Keyword            0
SERP features    607
Volume             0
URL                0
dtype: int64

In [13]:
df_10_000 = df.sample(n=10000, random_state=42)

In [14]:
df_10_000.to_csv('data/kw_10_000.csv', index=False)

In [15]:
df_1_000 = df_10_000.sample(n=1000, random_state=42)

In [16]:
df_1_000.to_csv('data/kw_1_000.csv', index=False)